In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../scrapers/data/requirements_txt.psv', sep='|',
                names=['repo', 'date', 'requirements'])

In [4]:
print df.shape
df = df.dropna() # I need to clean this data better. 
print df.shape

(107825, 3)
(105446, 3)


In [5]:
latest_data = df.groupby('repo').max()

In [6]:
latest_data.shape

(10135, 2)

In [9]:
from apriori import apriori

In [10]:
baskets = latest_data['requirements'].map(lambda s: s.split(',')).values

In [22]:
sorter = lambda r: r.bayesian_confidence/r.confidence*r.lift
filterer = lambda r: r['bayesian_confidence'] > 0.5
rules = apriori(baskets, 30, max_k=2, filter_rule=filterer, sort_key=sorter)

In [49]:
rules[130]

{'antecedent_occurrences': 40,
 'bayesian_confidence': 0.6482006399376371,
 'confidence': 0.775,
 'conviction': 4.4119936413967,
 'lift': 106.14358108108107,
 'occurrences': 31,
 'other_items': ['cssmin'],
 'start_items': ['jsmin'],
 'subsequent_occurrences': 74,
 'support': 0.003058707449432659}

In [32]:
score =  lambda r: r['bayesian_confidence'] * r['lift'] / r['confidence']

results = {}
for rule in rules:
    try:
        results[rule['start_items'][0]].append([
                                rule['other_items'][0], 
                                score(rule)             
                                        ])
    except KeyError:
        results[rule['start_items'][0]] = [[
                                rule['other_items'][0], 
                                score(rule)             
                                        ]]

In [33]:
import json

print json.dumps(results)

{"static3": [["django-toolbelt", 20.513976024674186], ["dj-static", 18.339954638996154], ["dj-database-url", 12.434641721195199], ["gunicorn", 6.1462562564034151], ["psycopg2", 5.8413610813032912], ["django", 3.5513865375297606]], "prettytable": [["six", 3.3297646320057761], ["requests", 2.7452456657999535]], "django-crispy-forms": [["django", 3.6137325983402064]], "distribute": [["wsgiref", 6.775848980433766], ["argparse", 6.4477953362429981]], "cffi": [["pycparser", 38.174588279226711], ["idna", 36.487395637212231], ["cryptography", 34.093401401529491], ["enum34", 24.087947934341219], ["pyasn1", 19.125389956624101], ["pyopenssl", 17.135085041902776], ["six", 3.9372928434716461], ["requests", 2.3174401843846169]], "ipython-genutils": [["pickleshare", 47.407354821908385], ["traitlets", 46.525297413364996], ["simplegeneric", 45.229190938394467], ["path", 42.922875311727857], ["pexpect", 38.279113491282395], ["gnureadline", 32.109978565796524], ["decorator", 26.985113253985908], ["ipytho